# Teaching markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
teaching = pd.read_excel('teaching.xlsx', 'teaching')
courses = pd.read_excel('teaching.xlsx', 'courses')

In [3]:
teaching.head(20)

,term,number,title,url_slug,venue,location
0,Fall 2007,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
1,Spring 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
2,Fall 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
3,Spring 2009,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
4,Fall 2009,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA"
5,Summer 2011,MATH 10,High School Algebra,class-2,Iowa State University,"Ames, IA"
6,Spring 2012,MATH 165,Calculus I,class-3,Iowa State University,"Ames, IA"
7,Summer 2012,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA"
8,Fall 2012,MATH 50,Calculus I,class-3,Drake Univeresity,"Des Moines, IA"
9,Spring 2013,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA"


In [4]:
teaching = teaching.merge(courses, how='left')
teaching

,term,number,title,url_slug,venue,location,description
0,Fall 2007,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring 2009,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall 2009,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
...,...,...,...,...,...,...,...
78,Fall 2021,FSEM 1010,First Year Seminar: Uses and Misuses of Algori...,class-23,Hamline University,"Saint Paul, MN",Data scientists have used algorithms for many ...
79,Winter 2022,MATH 1200,Statistics,class-20,Hamline University,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...
80,Spring 2022,CDS 1020,Introduction to Computational Data Science,class-27,Hamline University,"Saint Paul, MN",Goals: To continue the study of computational ...
81,Spring 2022,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...


In [5]:
teaching.head(20)

,term,number,title,url_slug,venue,location,description
0,Fall 2007,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring 2009,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall 2009,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
5,Summer 2011,MATH 10,High School Algebra,class-2,Iowa State University,"Ames, IA","Topics include signed numbers, polynomials, ra..."
6,Spring 2012,MATH 165,Calculus I,class-3,Iowa State University,"Ames, IA","Differential calculus, applications of the der..."
7,Summer 2012,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA","Integral calculus, applications of the integra..."
8,Fall 2012,MATH 50,Calculus I,class-3,Drake Univeresity,"Des Moines, IA","Very brief review of algebra, logarithms, and ..."
9,Spring 2013,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA","Integral calculus, applications of the integra..."


In [6]:
def term_order(term):
    if term == 'Fall':
        return '04'
    elif term in ['J-term', 'Winter']:
        return '01'
    elif term == 'Spring':
        return '02'
    elif term == 'Summer':
        return '03'

teaching['year'] = teaching.apply(lambda row: row.term.split(' ')[1], axis=1)
teaching['term'] = teaching.apply(lambda row: row.term.split(' ')[0], axis=1)
teaching['term_order'] = teaching.apply(lambda x: term_order(x.term), axis=1)

teaching = teaching[['term', 'year', 'term_order', 'number', 'title', 'url_slug', 'venue', 'location', 'description']]

In [7]:
teaching

,term,year,term_order,number,title,url_slug,venue,location,description
0,Fall,2007,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring,2008,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall,2008,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring,2009,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall,2009,04,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
...,...,...,...,...,...,...,...,...,...
78,Fall,2021,04,FSEM 1010,First Year Seminar: Uses and Misuses of Algori...,class-23,Hamline University,"Saint Paul, MN",Data scientists have used algorithms for many ...
79,Winter,2022,01,MATH 1200,Statistics,class-20,Hamline University,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...
80,Spring,2022,02,CDS 1020,Introduction to Computational Data Science,class-27,Hamline University,"Saint Paul, MN",Goals: To continue the study of computational ...
81,Spring,2022,02,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...


In [8]:
teaching.sort_values(by=['year', 'term_order', 'number'], axis=0, inplace=True)

In [9]:
def find_all_terms(course_number, venue):
    term_list = []
    for row, item in teaching.loc[(teaching.number == course_number) & (teaching.venue == venue)].iterrows():
        term_list.append(f'{item.term} {item.year}')

    return term_list

In [10]:
teaching['all_terms'] = teaching.apply(lambda row: find_all_terms(row.number, row.venue), axis=1)
teaching

,term,year,term_order,number,title,url_slug,venue,location,description,all_terms
0,Fall,2007,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]"
1,Spring,2008,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]"
2,Fall,2008,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]"
3,Spring,2009,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]"
4,Fall,2009,04,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ...",[Fall 2009]
...,...,...,...,...,...,...,...,...,...,...
77,Fall,2021,04,MATH 1200,Statistics,class-20,Hamline University,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...,"[Spring 2019, Summer 2019, Fall 2019, J-term 2..."
79,Winter,2022,01,MATH 1200,Statistics,class-20,Hamline University,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...,"[Spring 2019, Summer 2019, Fall 2019, J-term 2..."
80,Spring,2022,02,CDS 1020,Introduction to Computational Data Science,class-27,Hamline University,"Saint Paul, MN",Goals: To continue the study of computational ...,[Spring 2022]
81,Spring,2022,02,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...,"[Spring 2020, Spring 2021, Spring 2022]"


In [11]:
teaching.loc[teaching.venue == 'Grand View University']

,term,year,term_order,number,title,url_slug,venue,location,description,all_terms
12,Fall,2014,04,MATH 096,Introductory & Intermediate Algebra,class-6,Grand View University,"Des Moines, IA",This course covers: operations with numeric an...,"[Fall 2014, Spring 2015]"
13,Fall,2014,04,MATH 140,Introduction to Discrete Structures,class-7,Grand View University,"Des Moines, IA","This course is an introduction to set theory, ...",[Fall 2014]
14,Fall,2014,04,MATH 340,Discrete Computational Structures,class-8,Grand View University,"Des Moines, IA","Topics included are propositional logic, set t...","[Fall 2014, Fall 2016]"
15,Spring,2015,02,MATH 094,Introduction to Algebra,class-9,Grand View University,"Des Moines, IA",This course is a college preparatory course de...,"[Spring 2015, Summer 2015, Summer 2015, Summer..."
16,Spring,2015,02,MATH 096,Introductory & Intermediate Algebra,class-6,Grand View University,"Des Moines, IA",This course covers: operations with numeric an...,"[Fall 2014, Spring 2015]"
17,Spring,2015,02,MATH 351,Introduction to Numerical Analysis,class-10,Grand View University,"Des Moines, IA","Topics included are: error analysis, curve fit...",[Spring 2015]
18,Summer,2015,03,MATH 094,Introduction to Algebra,class-9,Grand View University,"Des Moines, IA",This course is a college preparatory course de...,"[Spring 2015, Summer 2015, Summer 2015, Summer..."
19,Summer,2015,03,MATH 094,Introduction to Algebra,class-9,Grand View University,"Des Moines, IA",This course is a college preparatory course de...,"[Spring 2015, Summer 2015, Summer 2015, Summer..."
20,Fall,2015,04,MATH 095,Intermediate Algebra,class-11,Grand View University,"Des Moines, IA",This course will include a brief review of: se...,[Fall 2015]
21,Fall,2015,04,MATH 116,Finite Mathematics,class-12,Grand View University,"Des Moines, IA","Topics include elementary linear functions, sy...","[Fall 2015, Fall 2015, Spring 2016, Spring 201..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [13]:
for row, item in teaching.iterrows():
    print(item)

term                                                        Fall
year                                                        2007
term_order                                                    04
number                                                  MATH 112
title                                            College Algebra
url_slug                                                 class-1
venue                                 Minnesota State University
location                                             Mankato, MN
description    Concepts of algebra (real numbers, exponents, ...
all_terms       [Fall 2007, Spring 2008, Fall 2008, Spring 2009]
Name: 0, dtype: object
term                                                      Spring
year                                                        2008
term_order                                                    02
number                                                  MATH 112
title                                            College Algebra
ur

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
loc_dict = {}

for row, item in teaching.iterrows():
    
    if '/' in str(item.number):
        tmp_number = item.number.replace('/', '')
        md_filename = str(tmp_number).replace(' ', '_') + '-' + item.url_slug + '.md'
        html_filename = str(tmp_number).replace(' ', '_') + "-" + item.url_slug 
    else:
        md_filename = str(item.number).replace(' ', '_') + "-" + item.url_slug + ".md"
        html_filename = str(item.number).replace(' ', '_') + "-" + item.url_slug 

    # md_filename = str(item.year) + "-" + str(item.term_order) + '-' + str(item.term) + '-' + item.url_slug + ".md"
    # html_filename = str(item.year) + "-" + str(item.term_order) + '-' + str(item.term) + '-'  + item.url_slug 

    year = item.year
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: teaching" + "\n"
    
    md += 'type: "teaching"\n'
    
    md += "permalink: /courses/" + html_filename + "\n"
        
    if len(str(item.url_slug)) > 3:
        md += "url_slug: " + str(item.url_slug) + "\n"

    # if len(str(item.term)) > 3:
    #     md += "term: " + str(item.term) + ' ' + str(item.year) + "\n"
    if len(str(item.all_terms)) > 3:
        md += "term: " + ', '.join(item.all_terms) + '\n'

    if len(str(item.number)) > 3:
        md += "number: " + str(item.number) + "\n"
        
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
#     if len(str(item.talk_url)) > 3:
#         md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_teaching/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [15]:
!ls ../_teaching

CDS_1010-class-22.md  MATH_112-class-1.md   MATH_300-class-13.md
CDS_1020-class-27.md  MATH_116-class-12.md  MATH_310-class-15.md
CDS_3200-class-24.md  MATH_1170-class-3.md  MATH_3320-class-18.md
CDS_5950-class-26.md  MATH_1200-class-20.md MATH_340-class-8.md
CSCI_1250-class-22.md MATH_140-class-1.md   MATH_3440-class-25.md
CSCI_1980-class-19.md MATH_140-class-7.md   MATH_351-class-10.md
FSEM_1010-class-23.md MATH_165-class-3.md   MATH_450-class-16.md
MATH_094-class-9.md   MATH_166-class-4.md   MATH_50-class-3.md
MATH_095-class-11.md  MATH_212-class-14.md  MATH_5950-class-21.md
MATH_096-class-6.md   MATH_261-class-17.md  MATH_70-class-4.md
MATH_10-class-2.md    MATH_2667-class-5.md


In [16]:
!cat ../_teaching/2012-03-Summer-class-4.md

cat: ../_teaching/2012-03-Summer-class-4.md: No such file or directory


In [17]:
!cat ../_teaching/CDS_1010-class-22.md

---
title: "Introduction to Programming"
collection: teaching
type: "teaching"
permalink: /courses/CDS_1010-class-22
url_slug: class-22
term: Spring 2020, Fall 2020, Fall 2020, Spring 2021, Fall 2021
number: CDS 1010
venue: "Hamline University"
location: "Saint Paul, MN"
---

Goals: To help students develop greater precision in their algorithmic thinking by writing moderate-sized programs for a variety of applications, including but not limited to biology, chemistry,  economics, literary studies, and mathematics.

Content: Students will learn the fundamentals of computer programming (loop structures, if-else statements, Boolean expressions, and arrays) to solve  problems from different disciplines. A short introduction to object-oriented programming is also given. This course is taught using Python.


In [18]:
!cat ../_teaching/FSEM_1010-class-23.md

---
title: "First Year Seminar: Uses and Misuses of Algorithms"
collection: teaching
type: "teaching"
permalink: /courses/FSEM_1010-class-23
url_slug: class-23
term: Fall 2019, Fall 2021
number: FSEM 1010
venue: "Hamline University"
location: "Saint Paul, MN"
---

Data scientists have used algorithms for many great things: Netflix’s recommendation system; building teams that go on to win the World Series or the Stanley Cup; proving the existence of the Higgs boson; and early detection of cancer. Other data scientists have used algorithms with malicious intent: targeting of vulnerable people by payday loan companies and for-profit higher education companies that provide little---if any---benefit to their students; using of social network bots to spread misinformation and sow discontent within a country. Sometimes algorithms have unintended negative effects: the firing of skilled teachers in Washington, D.C.; racial discrimination in the lengths of prison sentences and the granting (or n

In [19]:
# Create markdown table
sorted_teaching = teaching.sort_values(by=['year', 'term_order', 'number'], ascending=[False, False, True], axis=0)
sorted_teaching['course_url'] = sorted_teaching.apply(lambda row: 'https://cerickson30.github.io/courses/' + row.number.replace(' ', '_').replace('/', '') + '-' + row.url_slug, axis=1)
sorted_teaching

,term,year,term_order,number,title,url_slug,venue,location,description,all_terms,course_url
80,Spring,2022,02,CDS 1020,Introduction to Computational Data Science,class-27,Hamline University,"Saint Paul, MN",Goals: To continue the study of computational ...,[Spring 2022],https://cerickson30.github.io/courses/CDS_1020...
81,Spring,2022,02,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...,"[Spring 2020, Spring 2021, Spring 2022]",https://cerickson30.github.io/courses/CDS_3200...
82,Spring,2022,02,CDS 5950,Computational Data Science Capstone,class-26,Hamline University,"Saint Paul, MN",Goals: To help students integrate the knowledg...,"[Summer 2021, Spring 2022]",https://cerickson30.github.io/courses/CDS_5950...
79,Winter,2022,01,MATH 1200,Statistics,class-20,Hamline University,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...,"[Spring 2019, Summer 2019, Fall 2019, J-term 2...",https://cerickson30.github.io/courses/MATH_120...
75,Fall,2021,04,CDS 1010,Introduction to Programming,class-22,Hamline University,"Saint Paul, MN",Goals: To help students develop greater precis...,"[Spring 2020, Fall 2020, Fall 2020, Spring 202...",https://cerickson30.github.io/courses/CDS_1010...
...,...,...,...,...,...,...,...,...,...,...,...
4,Fall,2009,04,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ...",[Fall 2009],https://cerickson30.github.io/courses/MATH_140...
3,Spring,2009,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...
2,Fall,2008,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...
1,Spring,2008,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...


In [20]:
for thing in sorted_teaching.loc[sorted_teaching.year == "2013",].number:
    print(thing)

MATH 266/7
MATH 166


In [21]:
def make_venue_url(venue):
    if 'Hamline' in venue:
        url = 'https://www.hamline.edu'
    elif 'Grand View' in venue:
        url = 'https://www.grandview.edu'
    elif 'Drake' in venue:
        url = 'https://www.drake.edu'
    elif 'Iowa State' in venue:
        url = 'https://www.iastate.edu'
    elif 'Minnesota State' in venue:
        url = 'https://www.mnsu.edu'
    return f'[{venue}]({url})' + '{:target="_blank"}'

In [22]:
# sorted_teaching.number = sorted_teaching.apply(lambda row: f'<a href={row.course_url}>{row.number}</a>', axis=1)
sorted_teaching.number = sorted_teaching.apply(lambda row: f'[{row.number}]({row.course_url})', axis=1)
sorted_teaching.title = sorted_teaching.apply(lambda row: f'[{row.title}]({row.course_url})', axis=1)
sorted_teaching.venue = sorted_teaching.apply(lambda row: make_venue_url(row.venue), axis=1)
sorted_teaching['term_year'] = sorted_teaching.apply(lambda row: f'{row.term} {row.year}', axis=1)
sorted_teaching

,term,year,term_order,number,title,url_slug,venue,location,description,all_terms,course_url,term_year
80,Spring,2022,02,[CDS 1020](https://cerickson30.github.io/cours...,[Introduction to Computational Data Science](h...,class-27,[Hamline University](https://www.hamline.edu){...,"Saint Paul, MN",Goals: To continue the study of computational ...,[Spring 2022],https://cerickson30.github.io/courses/CDS_1020...,Spring 2022
81,Spring,2022,02,[CDS 3200](https://cerickson30.github.io/cours...,[Elements of Statistical Learning](https://cer...,class-24,[Hamline University](https://www.hamline.edu){...,"Saint Paul, MN",Goals: This is a continuation course for MATH ...,"[Spring 2020, Spring 2021, Spring 2022]",https://cerickson30.github.io/courses/CDS_3200...,Spring 2022
82,Spring,2022,02,[CDS 5950](https://cerickson30.github.io/cours...,[Computational Data Science Capstone](https://...,class-26,[Hamline University](https://www.hamline.edu){...,"Saint Paul, MN",Goals: To help students integrate the knowledg...,"[Summer 2021, Spring 2022]",https://cerickson30.github.io/courses/CDS_5950...,Spring 2022
79,Winter,2022,01,[MATH 1200](https://cerickson30.github.io/cour...,[Statistics](https://cerickson30.github.io/cou...,class-20,[Hamline University](https://www.hamline.edu){...,"Saint Paul, MN",Goals: To cover the fundamentals of statistica...,"[Spring 2019, Summer 2019, Fall 2019, J-term 2...",https://cerickson30.github.io/courses/MATH_120...,Winter 2022
75,Fall,2021,04,[CDS 1010](https://cerickson30.github.io/cours...,[Introduction to Programming](https://cerickso...,class-22,[Hamline University](https://www.hamline.edu){...,"Saint Paul, MN",Goals: To help students develop greater precis...,"[Spring 2020, Fall 2020, Fall 2020, Spring 202...",https://cerickson30.github.io/courses/CDS_1010...,Fall 2021
...,...,...,...,...,...,...,...,...,...,...,...,...
4,Fall,2009,04,[MATH 140](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,class-1,[Iowa State University](https://www.iastate.ed...,"Ames, IA","Coordinate geometry, quadratic and polynomial ...",[Fall 2009],https://cerickson30.github.io/courses/MATH_140...,Fall 2009
3,Spring,2009,02,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,class-1,[Minnesota State University](https://www.mnsu....,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...,Spring 2009
2,Fall,2008,04,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,class-1,[Minnesota State University](https://www.mnsu....,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...,Fall 2008
1,Spring,2008,02,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,class-1,[Minnesota State University](https://www.mnsu....,"Mankato, MN","Concepts of algebra (real numbers, exponents, ...","[Fall 2007, Spring 2008, Fall 2008, Spring 2009]",https://cerickson30.github.io/courses/MATH_112...,Spring 2008


In [23]:
courses_table = sorted_teaching[['term_year', 'number', 'title', 'venue']]

In [24]:
courses_table

,term_year,number,title,venue
80,Spring 2022,[CDS 1020](https://cerickson30.github.io/cours...,[Introduction to Computational Data Science](h...,[Hamline University](https://www.hamline.edu){...
81,Spring 2022,[CDS 3200](https://cerickson30.github.io/cours...,[Elements of Statistical Learning](https://cer...,[Hamline University](https://www.hamline.edu){...
82,Spring 2022,[CDS 5950](https://cerickson30.github.io/cours...,[Computational Data Science Capstone](https://...,[Hamline University](https://www.hamline.edu){...
79,Winter 2022,[MATH 1200](https://cerickson30.github.io/cour...,[Statistics](https://cerickson30.github.io/cou...,[Hamline University](https://www.hamline.edu){...
75,Fall 2021,[CDS 1010](https://cerickson30.github.io/cours...,[Introduction to Programming](https://cerickso...,[Hamline University](https://www.hamline.edu){...
...,...,...,...,...
4,Fall 2009,[MATH 140](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,[Iowa State University](https://www.iastate.ed...
3,Spring 2009,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,[Minnesota State University](https://www.mnsu....
2,Fall 2008,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,[Minnesota State University](https://www.mnsu....
1,Spring 2008,[MATH 112](https://cerickson30.github.io/cours...,[College Algebra](https://cerickson30.github.i...,[Minnesota State University](https://www.mnsu....


In [25]:
courses_table.number[0]

'[MATH 112](https://cerickson30.github.io/courses/MATH_112-class-1)'

In [26]:
# storing the content of csv file in a list_of_rows. Each row is a dict.
list_of_rows = [dict_row[1] for dict_row in courses_table.iterrows()]

# For Headers of the csv file.
headers = list(list_of_rows[0].keys())

md_string = """---
layout: archive
title: "Teaching"
permalink: /courses/
author_profile: true
redirect_from:
  - /teaching
---"""

# The below code block makes md_string as per the required format of a markdown file.
# md_string = " | "
# for header in headers:
#     md_string += header+" |"
md_string += '\n\n\n<br>'
md_string += "\n\n\n |"
for i in range(len(headers)):
    md_string += "--- | "

md_string += "\n"
for row in list_of_rows:
    md_string += " | "
    for header in headers:
        md_string += row[header]+" | "
    md_string += "\n"

In [27]:
with open('../_pages/teaching.md', 'w') as outfile:
    outfile.write(md_string)

with open('../_pages/courses.md', 'w') as outfile:
    outfile.write(md_string)

In [28]:
# my_str = ''
# for row in list_of_rows:
#     my_str += ' | '
#     for header in headers:
#         my_str += row[header] + ' | '
#     my_str += '\n'
#     print(my_str, end='')